In [0]:
import time
import mlflow
run_name = f"LogReg_{int(time.time())}"  # 예: LogReg_1685981234
mlflow.set_experiment('/MLflow')
# MLflow 실행(run) 시작
with mlflow.start_run(run_name=run_name):
    # Log a test parameter
    mlflow.log_param("test_param", "test_value")

In [0]:
# Databricks Notebook에 붙여넣고 실행하세요.

# 1. 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, count, avg, desc, udf
from pyspark.sql.types import ArrayType, StringType, IntegerType

# Spark 세션 초기화 (Databricks에서는 자동으로 초기화되지만 명시적으로 정의 가능)
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# 2. 데이터 로드 (Unity Catalog 테이블에서 불러오도록 변경)
# '1dt_team8_managed.movielens-small' 데이터베이스에 movies, ratings 테이블이 있다고 가정합니다.
try:
    movies_df_spark = spark.read.table("`1dt_team8_managed`.`movielens-small`.movies")
    ratings_df_spark = spark.read.table("`1dt_team8_managed`.`movielens-small`.ratings")
    print("데이터를 Unity Catalog 테이블에서 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"Unity Catalog 테이블 로드 중 오류 발생: {e}")
    print("기존 경로 변수 방식으로 시도합니다. (데이터 경로를 다시 확인해주세요)")
    movies_path = "/databricks-datasets/movielens/small/movies.csv"
    ratings_path = "/databricks-datasets/movielens/small/ratings.csv"
    movies_df_spark = spark.read.csv(movies_path, header=True, inferSchema=True)
    ratings_df_spark = spark.read.csv(ratings_path, header=True, inferSchema=True)


# Pandas DataFrame으로 변환 (콘텐츠 기반 추천을 위해 TF-IDF를 사용하기 위함)
# 대규모 데이터셋의 경우 이 변환 대신 Spark MLlib 사용을 고려해야 합니다.
movies_df = movies_df_spark.toPandas()
ratings_df = ratings_df_spark.toPandas()

# 데이터 확인
# print("Movies DataFrame head:")
# print(movies_df.head())
# print("\nRatings DataFrame head:")
# print(ratings_df.head())

# 3. 데이터 전처리 및 콘텐츠 기반 추천 시스템 구축

# 장르 데이터 클리닝 및 TF-IDF 벡터화
# genres 컬럼의 누락된 값 처리 (결측치는 빈 문자열로 대체)
movies_df['genres'] = movies_df['genres'].fillna('')

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres'])

print(f"\nTF-IDF matrix shape: {tfidf_matrix.shape}")

# 코사인 유사도 계산 (콘텐츠 기반 유사도)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화 제목과 인덱스를 매핑하는 시리즈 생성
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

# 영화 ID로 제목을 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_title_from_id(movie_id):
    if movie_id in movies_df['movieId'].values:
        return movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]
    return None

# 영화 이름으로 ID를 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_movie_ids_from_names(movie_names):
    found_ids = []
    not_found_names = []
    for name in movie_names:
        # 정확히 일치하는 제목 찾기
        match = movies_df[movies_df['title'].str.lower() == name.lower()]
        if not match.empty:
            found_ids.append(match['movieId'].iloc[0])
        else:
            not_found_names.append(name)
    return found_ids, not_found_names


# 4. 추천 함수 정의

def get_content_based_recommendations(movie_id, cosine_sim_matrix, movies_df_local, indices_local, num_recommendations=10):
    """
    콘텐츠(장르) 기반으로 유사한 영화를 추천합니다. (movie_id 기반)
    """
    movie_title = get_title_from_id(movie_id)
    if not movie_title or movie_title not in indices_local:
        return pd.DataFrame()

    idx = indices_local[movie_title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 자기 자신 제외
    sim_scores = sim_scores[1:num_recommendations + 1]
    movie_indices = [i[0] for i in sim_scores]
    return movies_df_local.iloc[movie_indices]


def recommend_movies_by_input(user_movie_ids, num_recommendations_total=10, num_popular_or_high_rated_fixed=1):
    """
    사용자가 입력한 영화 ID를 기반으로 콘텐츠 기반 추천 + 인기/고평점 영화를 섞어서 추천합니다.
    num_popular_or_high_rated_fixed 만큼 리뷰 많고 평점 높은 영화를 무조건 포함합니다.
    """
    recommended_movies = pd.DataFrame()
    seen_movie_ids = set(user_movie_ids) # 입력 영화는 추천 목록에서 제외

    # --- 인기/고평점 영화 목록 미리 계산 ---
    movie_stats_df_spark = ratings_df_spark.groupBy("movieId").agg(
        count(col("rating")).alias("num_ratings"),
        avg(col("rating")).alias("avg_rating")
    )

    min_ratings_threshold = 50 # 최소 리뷰 수 임계값
    popular_movies_spark_filtered = movie_stats_df_spark.filter(col("num_ratings") >= min_ratings_threshold) \
                                                        .orderBy(desc("avg_rating"), desc("num_ratings")) \
                                                        .join(movies_df_spark, "movieId", "left")
    popular_movies_df = popular_movies_spark_filtered.toPandas()

    # --- 1. 리뷰 많고 평점 높은 영화를 지정된 개수만큼 무조건 포함 ---
    popular_added_count = 0
    for _, row in popular_movies_df.iterrows():
        if row['movieId'] not in seen_movie_ids:
            recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
            seen_movie_ids.add(row['movieId'])
            popular_added_count += 1
            if popular_added_count >= num_popular_or_high_rated_fixed:
                break
    
    # --- 2. 나머지 추천 개수 (총 num_recommendations_total 중 인기영화 제외) ---
    remaining_recommendations_slots = num_recommendations_total - popular_added_count
    
    # 남은 슬롯이 없거나 음수가 되면 더 이상 콘텐츠 기반 추천을 할 필요 없음
    if remaining_recommendations_slots <= 0:
        # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
        final_recommendations = recommended_movies[~recommended_movies['movieId'].isin(user_movie_ids)] \
                                .drop_duplicates(subset=['movieId']) \
                                .head(num_recommendations_total) \
                                .reset_index(drop=True)
        return spark.createDataFrame(final_recommendations)


    # --- 3. 콘텐츠 기반 추천으로 나머지 채우기 ---
    # 각 입력 영화에 대한 콘텐츠 기반 추천을 가져와 합치기
    for movie_id in user_movie_ids:
        # 필요한 remaining_recommendations_slots보다 더 많이 가져와서 중복 및 이미 추가된 인기영화 제거
        content_recs_for_one_movie = get_content_based_recommendations(
            movie_id, cosine_sim, movies_df, indices, remaining_recommendations_slots * 2
        )
        for _, row in content_recs_for_one_movie.iterrows():
            if row['movieId'] not in seen_movie_ids:
                recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                # 필요한 콘텐츠 기반 추천 개수를 채웠으면 중단
                if len(recommended_movies) - popular_added_count >= remaining_recommendations_slots:
                    break
        # 전체 콘텐츠 기반 추천 개수를 채웠으면 더 이상 다른 입력 영화를 보지 않아도 됨
        if len(recommended_movies) - popular_added_count >= remaining_recommendations_slots:
            break

    # --- 4. 여전히 추천 개수가 부족하면 추가 인기/고평점 영화로 채우기 ---
    if len(recommended_movies) < num_recommendations_total:
        additional_popular_needed = num_recommendations_total - len(recommended_movies)
        added_count = 0
        for _, row in popular_movies_df.iterrows():
            if row['movieId'] not in seen_movie_ids:
                recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                added_count += 1
                if added_count >= additional_popular_needed:
                    break

    # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
    final_recommendations = recommended_movies[~recommended_movies['movieId'].isin(user_movie_ids)] \
                            .drop_duplicates(subset=['movieId']) \
                            .head(num_recommendations_total) \
                            .reset_index(drop=True)

    # Spark DataFrame으로 변환하여 반환 (Databricks display 호환성을 위해)
    return spark.createDataFrame(final_recommendations)


# 5. 모델 테스트 (사용자 입력)

# 사용자에게 영화 이름 또는 ID 입력 요청
num_recommendations_output = 20 # 최종 추천할 영화 개수
num_guaranteed_popular = 5 # 추천 목록에 포함할 인기 영화 개수 조절

while True:
    user_input_str = input("추천받고 싶은 영화 이름 또는 ID를 쉼표로 구분하여 3개 이상 입력하세요 (예: Toy Story, Jumanji, Braveheart 또는 1,2,13): ")
    
    is_id_input = True
    processed_inputs = []
    for item in user_input_str.split(','):
        stripped_item = item.strip()
        try:
            processed_inputs.append(int(stripped_item))
        except ValueError:
            is_id_input = False
            processed_inputs.append(stripped_item)

    if not (3 <= len(processed_inputs)):
        print("영화는 3개 이상 입력해야 합니다. 다시 입력해주세요.")
        continue

    input_movie_ids = []
    input_movie_names_display = []

    if is_id_input:
        input_movie_ids = processed_inputs
        found_movie_info = movies_df_spark.filter(col("movieId").isin(input_movie_ids)).select("movieId", "title").collect()
        found_ids_set = {row.movieId for row in found_movie_info}
        
        if len(found_ids_set) != len(input_movie_ids):
            missing_ids = set(input_movie_ids) - found_ids_set
            print(f"오류: 다음 영화 ID를 찾을 수 없습니다: {list(missing_ids)}. 다시 입력해주세요.")
            continue
        
        movie_id_to_title = {row.movieId: row.title for row in found_movie_info}
        input_movie_names_display = [movie_id_to_title[mid] for mid in input_movie_ids]
        print(f"\n입력된 영화 ID: {input_movie_ids}")

    else: # 이름으로 입력된 경우
        input_movie_names = processed_inputs
        # Pandas DataFrame 기반의 get_movie_ids_from_names 함수 사용
        input_movie_ids, not_found_names = get_movie_ids_from_names(input_movie_names)
        
        if not_found_names:
            print(f"다음 영화를 찾을 수 없거나 모호하여 추천을 진행할 수 없습니다: {', '.join(not_found_names)}")
            continue
        elif len(input_movie_ids) != len(input_movie_names): # 찾았으나 개수가 다르면 문제
            print("입력된 모든 영화 이름을 영화 ID로 변환하지 못했습니다. 다시 시도해주세요.")
            continue
        
        input_movie_names_display = input_movie_names

    break # 유효한 입력이 들어왔으므로 루프 종료

print(f"\n입력된 영화: {input_movie_names_display} (ID: {input_movie_ids})를 기반으로 영화를 추천합니다:")
# recommend_movies_by_input 함수는 Spark DataFrame을 반환
# num_guaranteed_popular 변수를 사용하여 인기 영화 개수 조절
recommended_df = recommend_movies_by_input(input_movie_ids, num_recommendations_output, num_popular_or_high_rated_fixed=num_guaranteed_popular)
# Databricks의 display 함수를 사용하여 출력
display(recommended_df.select('title', 'genres'))


# 6. 모델 성능 평가 및 시각화

# 평가 지표 계산 함수 (간단한 추천 시스템의 Precision, Recall 계산)
def evaluate_recommender(model_func, movies_df_eval, ratings_df_eval, indices_eval, cosine_sim_matrix_eval,
                         test_ratings_df, num_recommendations_total=10, num_popular_fixed=1):
    """
    간단한 Precision, Recall 계산 (모든 사용자에 대해 평가하기는 어려우므로 샘플링)
    """
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    num_users_evaluated = 0

    # 테스트 데이터셋에서 임의의 사용자 샘플링
    sample_users = test_ratings_df['userId'].sample(min(50, test_ratings_df['userId'].nunique()), random_state=42).unique()

    for user_id in sample_users:
        # 해당 사용자의 훈련 데이터에서 가장 높은 평점을 준 영화들을 '입력 영화'로 가정
        user_train_movies = train_ratings[train_ratings['userId'] == user_id].sort_values(by='rating', ascending=False)
        
        # 훈련 데이터에서 최소 3개 이상의 영화를 본 사용자만 고려
        if len(user_train_movies) < 3:
            continue
            
        # 가장 높은 평점을 준 상위 3개 영화의 ID를 시드(seed) 영화로 사용
        seed_movie_ids = user_train_movies.head(3)['movieId'].tolist()

        if not seed_movie_ids:
            continue

        # 추천 받기
        # recommend_movies_by_input 함수는 Spark DataFrame을 반환하므로 .toPandas()로 변환
        recommended_movies_df_spark = model_func(
            seed_movie_ids, num_recommendations_total, num_popular_or_high_rated_fixed=num_popular_fixed
        )
        recommended_movies_df = recommended_movies_df_spark.toPandas()
        
        if recommended_movies_df.empty:
            continue

        recommended_movie_ids = set(recommended_movies_df['movieId'].tolist())

        # 해당 사용자가 테스트 데이터에서 실제로 높은 평점을 준 영화 목록 (예: 4점 이상)
        actual_relevant_movies_df = test_ratings_df[(test_ratings_df['userId'] == user_id) & (test_ratings_df['rating'] >= 4)]
        actual_relevant_movie_ids = set(actual_relevant_movies_df['movieId'].tolist())

        # 교집합 (추천된 영화 중 실제 관련성 있는 영화)
        true_positives = len(recommended_movie_ids.intersection(actual_relevant_movie_ids))

        # Precision: 추천된 영화 중 실제로 관련성 있는 영화의 비율
        precision = true_positives / len(recommended_movie_ids) if len(recommended_movie_ids) > 0 else 0

        # Recall: 실제 관련성 있는 영화 중 추천된 영화의 비율
        recall = true_positives / len(actual_relevant_movie_ids) if len(actual_relevant_movie_ids) > 0 else 0

        # F1-score: Precision과 Recall의 조화 평균
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        total_precision += precision
        total_recall += recall
        total_f1 += f1
        num_users_evaluated += 1

    if num_users_evaluated == 0:
        return 0, 0, 0, 0 # 평가된 사용자 없음

    avg_precision = total_precision / num_users_evaluated
    avg_recall = total_recall / num_users_evaluated
    avg_f1 = total_f1 / num_users_evaluated

    return avg_precision, avg_recall, avg_f1, num_users_evaluated

# 평가를 위한 데이터 분할 (사용자-영화 상호작용 데이터)
# ratings_df를 train/test로 분할
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.2, random_state=42)

# 평가 실행 함수
def model_performance_evaluation():
    print("\n모델 성능 평가를 시작합니다...")
    # evaluate_recommender 호출 시 num_popular_fixed 변수를 전달하여 조절 가능
    avg_precision, avg_recall, avg_f1, num_evaluated_users = evaluate_recommender(
        recommend_movies_by_input, movies_df, ratings_df, indices, cosine_sim, test_ratings, num_popular_fixed=num_guaranteed_popular
    )

    print(f"\n평가된 사용자 수: {num_evaluated_users}")
    print(f"평균 Precision: {avg_precision:.4f}")
    print(f"평균 Recall: {avg_recall:.4f}")
    print(f"평균 F1-Score: {avg_f1:.4f}")

    # 성능 시각화
    metrics = ['Precision', 'Recall', 'F1-Score']
    values = [avg_precision, avg_recall, avg_f1]

    plt.figure(figsize=(8, 6))
    sns.barplot(x=metrics, y=values, palette='viridis')
    plt.title('Recommendation System Performance Metrics')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom', fontsize=10)
    plt.show()

# 모델 성능 평가 함수 호출
#model_performance_evaluation()

In [0]:
# Databricks Notebook에 붙여넣고 실행하세요.

# 1. 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, count, avg, desc, udf
from pyspark.sql.types import ArrayType, StringType, IntegerType

# Spark 세션 초기화 (Databricks에서는 자동으로 초기화되지만 명시적으로 정의 가능)
spark = SparkSession.builder.appName("MovieRecommendation").getOrCreate()

# 2. 데이터 로드 (Unity Catalog 테이블에서 불러오도록 변경)
# '1dt_team8_managed.movielens-small' 데이터베이스에 movies, ratings 테이블이 있다고 가정합니다.
try:
    movies_df_spark = spark.read.table("`1dt_team8_managed`.`movielens-small`.movies")
    ratings_df_spark = spark.read.table("`1dt_team8_managed`.`movielens-small`.ratings")
    print("데이터를 Unity Catalog 테이블에서 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"Unity Catalog 테이블 로드 중 오류 발생: {e}")
    print("기존 경로 변수 방식으로 시도합니다. (데이터 경로를 다시 확인해주세요)")
    movies_path = "/databricks-datasets/movielens/small/movies.csv"
    ratings_path = "/databricks-datasets/movielens/small/ratings.csv"
    movies_df_spark = spark.read.csv(movies_path, header=True, inferSchema=True)
    ratings_df_spark = spark.read.csv(ratings_path, header=True, inferSchema=True)


# Pandas DataFrame으로 변환 (콘텐츠 기반 추천을 위해 TF-IDF를 사용하기 위함)
# 대규모 데이터셋의 경우 이 변환 대신 Spark MLlib 사용을 고려해야 합니다.
movies_df = movies_df_spark.toPandas()
ratings_df = ratings_df_spark.toPandas()

# 데이터 확인
# print("Movies DataFrame head:")
# print(movies_df.head())
# print("\nRatings DataFrame head:")
# print(ratings_df.head())

# 3. 데이터 전처리 및 콘텐츠 기반 추천 시스템 구축

# 장르 데이터 클리닝 및 TF-IDF 벡터화
# genres 컬럼의 누락된 값 처리 (결측치는 빈 문자열로 대체)
movies_df['genres'] = movies_df['genres'].fillna('')

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['genres'])

print(f"\nTF-IDF matrix shape: {tfidf_matrix.shape}")

# 코사인 유사도 계산 (콘텐츠 기반 유사도)
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 영화 제목과 인덱스를 매핑하는 시리즈 생성
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

# 영화 ID로 제목을 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_title_from_id(movie_id):
    if movie_id in movies_df['movieId'].values:
        return movies_df[movies_df['movieId'] == movie_id]['title'].iloc[0]
    return None

# 영화 이름으로 ID를 찾는 헬퍼 함수 (Pandas DataFrame 사용)
def get_movie_ids_from_names(movie_names):
    found_ids = []
    not_found_names = []
    for name in movie_names:
        # 정확히 일치하는 제목 찾기
        match = movies_df[movies_df['title'].str.lower() == name.lower()]
        if not match.empty:
            found_ids.append(match['movieId'].iloc[0])
        else:
            not_found_names.append(name)
    return found_ids, not_found_names


# 4. 추천 함수 정의

def get_content_based_recommendations(movie_id, cosine_sim_matrix, movies_df_local, indices_local, num_recommendations=10):
    """
    콘텐츠(장르) 기반으로 유사한 영화를 추천합니다. (movie_id 기반)
    """
    movie_title = get_title_from_id(movie_id)
    if not movie_title or movie_title not in indices_local:
        return pd.DataFrame()

    idx = indices_local[movie_title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 자기 자신 제외
    sim_scores = sim_scores[1:num_recommendations + 1]
    movie_indices = [i[0] for i in sim_scores]
    return movies_df_local.iloc[movie_indices]


def recommend_movies_by_input(user_movie_ids, num_popular_or_high_rated_fixed, num_recommendations_total=10):
    """
    사용자가 입력한 영화 ID를 기반으로 콘텐츠 기반 추천 + 인기/고평점 영화를 섞어서 추천합니다.
    num_popular_or_high_rated_fixed 만큼 리뷰 많고 평점 높은 영화를 무조건 포함합니다.
    """
    recommended_movies = pd.DataFrame()
    seen_movie_ids = set(user_movie_ids) # 입력 영화는 추천 목록에서 제외

    # --- 인기/고평점 영화 목록 미리 계산 ---
    movie_stats_df_spark = ratings_df_spark.groupBy("movieId").agg(
        count(col("rating")).alias("num_ratings"),
        avg(col("rating")).alias("avg_rating")
    )

    min_ratings_threshold = 50 # 최소 리뷰 수 임계값
    popular_movies_spark_filtered = movie_stats_df_spark.filter(col("num_ratings") >= min_ratings_threshold) \
                                                        .orderBy(desc("avg_rating"), desc("num_ratings")) \
                                                        .join(movies_df_spark, "movieId", "left")
    popular_movies_df = popular_movies_spark_filtered.toPandas()

    # --- 1. 리뷰 많고 평점 높은 영화를 지정된 개수만큼 무조건 포함 ---
    popular_added_count = 0
    for _, row in popular_movies_df.iterrows():
        if row['movieId'] not in seen_movie_ids:
            recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
            seen_movie_ids.add(row['movieId'])
            popular_added_count += 1
            if popular_added_count >= num_popular_or_high_rated_fixed:
                break
    
    # --- 2. 나머지 추천 개수 (총 num_recommendations_total 중 인기영화 제외) ---
    remaining_recommendations_slots = num_recommendations_total - popular_added_count
    
    # 남은 슬롯이 없거나 음수가 되면 더 이상 콘텐츠 기반 추천을 할 필요 없음
    if remaining_recommendations_slots <= 0:
        # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
        final_recommendations = recommended_movies[~recommended_movies['movieId'].isin(user_movie_ids)] \
                                .drop_duplicates(subset=['movieId']) \
                                .head(num_recommendations_total) \
                                .reset_index(drop=True)
        return spark.createDataFrame(final_recommendations)


    # --- 3. 콘텐츠 기반 추천으로 나머지 채우기 ---
    # 각 입력 영화에 대한 콘텐츠 기반 추천을 가져와 합치기
    for movie_id in user_movie_ids:
        # 필요한 remaining_recommendations_slots보다 더 많이 가져와서 중복 및 이미 추가된 인기영화 제거
        content_recs_for_one_movie = get_content_based_recommendations(
            movie_id, cosine_sim, movies_df, indices, remaining_recommendations_slots * 2
        )
        for _, row in content_recs_for_one_movie.iterrows():
            if row['movieId'] not in seen_movie_ids:
                recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                # 필요한 콘텐츠 기반 추천 개수를 채웠으면 중단
                if len(recommended_movies) - popular_added_count >= remaining_recommendations_slots:
                    break
        # 전체 콘텐츠 기반 추천 개수를 채웠으면 더 이상 다른 입력 영화를 보지 않아도 됨
        if len(recommended_movies) - popular_added_count >= remaining_recommendations_slots:
            break

    # --- 4. 여전히 추천 개수가 부족하면 추가 인기/고평점 영화로 채우기 ---
    if len(recommended_movies) < num_recommendations_total:
        additional_popular_needed = num_recommendations_total - len(recommended_movies)
        added_count = 0
        for _, row in popular_movies_df.iterrows():
            if row['movieId'] not in seen_movie_ids:
                recommended_movies = pd.concat([recommended_movies, pd.DataFrame([row])], ignore_index=True)
                seen_movie_ids.add(row['movieId'])
                added_count += 1
                if added_count >= additional_popular_needed:
                    break

    # 최종 추천 목록에서 필요한 개수만큼 자르기 (중복 및 입력 영화 제거 후)
    final_recommendations = recommended_movies[~recommended_movies['movieId'].isin(user_movie_ids)] \
                            .drop_duplicates(subset=['movieId']) \
                            .head(num_recommendations_total) \
                            .reset_index(drop=True)

    # Spark DataFrame으로 변환하여 반환 (Databricks display 호환성을 위해)
    return spark.createDataFrame(final_recommendations)


# 5. 모델 테스트 (사용자 입력)

# 사용자에게 영화 이름 또는 ID 입력 요청
num_recommendations_output = 20 # 최종 추천할 영화 개수
num_guaranteed_popular = 10 # 추천 목록에 포함할 인기 영화 개수 조절

while True:
    user_input_str = input("추천받고 싶은 영화 이름 또는 ID를 쉼표로 구분하여 3~5개 입력하세요 (예: Toy Story, Jumanji, Braveheart 또는 1,2,13): ")
    
    is_id_input = True
    processed_inputs = []
    for item in user_input_str.split(','):
        stripped_item = item.strip()
        try:
            processed_inputs.append(int(stripped_item))
        except ValueError:
            is_id_input = False
            processed_inputs.append(stripped_item)

    if not (3 <= len(processed_inputs) <= 5):
        print("영화는 3개에서 5개 사이로 입력해야 합니다. 다시 입력해주세요.")
        continue

    input_movie_ids = []
    input_movie_names_display = []

    if is_id_input:
        input_movie_ids = processed_inputs
        found_movie_info = movies_df_spark.filter(col("movieId").isin(input_movie_ids)).select("movieId", "title").collect()
        found_ids_set = {row.movieId for row in found_movie_info}
        
        if len(found_ids_set) != len(input_movie_ids):
            missing_ids = set(input_movie_ids) - found_ids_set
            print(f"오류: 다음 영화 ID를 찾을 수 없습니다: {list(missing_ids)}. 다시 입력해주세요.")
            continue
        
        movie_id_to_title = {row.movieId: row.title for row in found_movie_info}
        input_movie_names_display = [movie_id_to_title[mid] for mid in input_movie_ids]
        print(f"\n입력된 영화 ID: {input_movie_ids}")

    else: # 이름으로 입력된 경우
        input_movie_names = processed_inputs
        # Pandas DataFrame 기반의 get_movie_ids_from_names 함수 사용
        input_movie_ids, not_found_names = get_movie_ids_from_names(input_movie_names)
        
        if not_found_names:
            print(f"다음 영화를 찾을 수 없거나 모호하여 추천을 진행할 수 없습니다: {', '.join(not_found_names)}")
            continue
        elif len(input_movie_ids) != len(input_movie_names): # 찾았으나 개수가 다르면 문제
            print("입력된 모든 영화 이름을 영화 ID로 변환하지 못했습니다. 다시 시도해주세요.")
            continue
        
        input_movie_names_display = input_movie_names

    break # 유효한 입력이 들어왔으므로 루프 종료

print(f"\n입력된 영화: {input_movie_names_display} (ID: {input_movie_ids})를 기반으로 영화를 추천합니다:")
# recommend_movies_by_input 함수는 Spark DataFrame을 반환
# num_guaranteed_popular 변수를 사용하여 인기 영화 개수 조절
recommended_df = recommend_movies_by_input(input_movie_ids, num_recommendations_output, num_guaranteed_popular)
# Databricks의 display 함수를 사용하여 출력
display(recommended_df.select('title', 'genres'))


# 6. 모델 성능 평가 및 시각화

# 평가 지표 계산 함수 (간단한 추천 시스템의 Precision, Recall 계산)
def evaluate_recommender(model_func, movies_df_eval, ratings_df_eval, indices_eval, cosine_sim_matrix_eval,
                         test_ratings_df, num_recommendations_total=10, num_popular_fixed=1):
    """
    간단한 Precision, Recall 계산 (모든 사용자에 대해 평가하기는 어려우므로 샘플링)
    """
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    num_users_evaluated = 0

    # 테스트 데이터셋에서 임의의 사용자 샘플링
    sample_users = test_ratings_df['userId'].sample(min(50, test_ratings_df['userId'].nunique()), random_state=42).unique()

    for user_id in sample_users:
        # 해당 사용자의 훈련 데이터에서 가장 높은 평점을 준 영화들을 '입력 영화'로 가정
        user_train_movies = train_ratings[train_ratings['userId'] == user_id].sort_values(by='rating', ascending=False)
        
        # 훈련 데이터에서 최소 3개 이상의 영화를 본 사용자만 고려
        if len(user_train_movies) < 3:
            continue
            
        # 가장 높은 평점을 준 상위 3개 영화의 ID를 시드(seed) 영화로 사용
        seed_movie_ids = user_train_movies.head(3)['movieId'].tolist()

        if not seed_movie_ids:
            continue

        # 추천 받기
        # recommend_movies_by_input 함수는 Spark DataFrame을 반환하므로 .toPandas()로 변환
        recommended_movies_df_spark = model_func(
            seed_movie_ids, num_recommendations_total, num_popular_or_high_rated_fixed=num_popular_fixed
        )
        recommended_movies_df = recommended_movies_df_spark.toPandas()
        
        if recommended_movies_df.empty:
            continue

        recommended_movie_ids = set(recommended_movies_df['movieId'].tolist())

        # 해당 사용자가 테스트 데이터에서 실제로 높은 평점을 준 영화 목록 (예: 4점 이상)
        actual_relevant_movies_df = test_ratings_df[(test_ratings_df['userId'] == user_id) & (test_ratings_df['rating'] >= 4)]
        actual_relevant_movie_ids = set(actual_relevant_movies_df['movieId'].tolist())

        # 교집합 (추천된 영화 중 실제 관련성 있는 영화)
        true_positives = len(recommended_movie_ids.intersection(actual_relevant_movie_ids))

        # Precision: 추천된 영화 중 실제로 관련성 있는 영화의 비율
        precision = true_positives / len(recommended_movie_ids) if len(recommended_movie_ids) > 0 else 0

        # Recall: 실제 관련성 있는 영화 중 추천된 영화의 비율
        recall = true_positives / len(actual_relevant_movie_ids) if len(actual_relevant_movie_ids) > 0 else 0

        # F1-score: Precision과 Recall의 조화 평균
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        total_precision += precision
        total_recall += recall
        total_f1 += f1
        num_users_evaluated += 1

    if num_users_evaluated == 0:
        return 0, 0, 0, 0 # 평가된 사용자 없음

    avg_precision = total_precision / num_users_evaluated
    avg_recall = total_recall / num_users_evaluated
    avg_f1 = total_f1 / num_users_evaluated

    return avg_precision, avg_recall, avg_f1, num_users_evaluated

# 평가를 위한 데이터 분할 (사용자-영화 상호작용 데이터)
# ratings_df를 train/test로 분할
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.2, random_state=42)

# 평가 실행 함수
def model_performance_evaluation():
    print("\n모델 성능 평가를 시작합니다...")
    # evaluate_recommender 호출 시 num_popular_fixed 변수를 전달하여 조절 가능
    avg_precision, avg_recall, avg_f1, num_evaluated_users = evaluate_recommender(
        recommend_movies_by_input, movies_df, ratings_df, indices, cosine_sim, test_ratings, num_popular_fixed=num_guaranteed_popular
    )

    print(f"\n평가된 사용자 수: {num_evaluated_users}")
    print(f"평균 Precision: {avg_precision:.4f}")
    print(f"평균 Recall: {avg_recall:.4f}")
    print(f"평균 F1-Score: {avg_f1:.4f}")

    # 성능 시각화
    metrics = ['Precision', 'Recall', 'F1-Score']
    values = [avg_precision, avg_recall, avg_f1]

    plt.figure(figsize=(8, 6))
    sns.barplot(x=metrics, y=values, palette='viridis')
    plt.title('Recommendation System Performance Metrics')
    plt.ylabel('Score')
    plt.ylim(0, 1)
    for i, v in enumerate(values):
        plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom', fontsize=10)
    plt.show()

# 모델 성능 평가 함수 호출
# model_performance_evaluation()